In [1]:
import sqlite3
import os
import csv
import pandas as pd

## Functions to Upload Data to Database Tables

In [2]:
# This fuction gets column_names and produces an insert statement to insert values to the database

def prep_insert_statement(table_name):
    # Execute PRAGMA table_info() for the specified table
    cursor.execute(f"PRAGMA table_info({table_name});")
    
    # Fetch all rows from the executed query
    columns_info = cursor.fetchall()
    
    # Extract the column names from the columns_info
    column_names = [column[1] for column in columns_info]
    
    # Prepare column names for the SQL statement
    columns_str = ', '.join(column_names)  # Column names as a comma-separated string
    
    # Prepare placeholders for the values
    placeholders = ', '.join(['?' for _ in column_names])  # A placeholder for each column
    
    # Construct the INSERT INTO statement
    insert_stmt = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders});"
    
    return column_names, insert_stmt

In [3]:
# This function takes a csv file and writes the data to the selected database table

def insert_data_to_tables(table_name, file_name, file_extension='csv'):
    column_names, insert_stmt = prep_insert_statement(table_name)
    
    # Upload JOBS data to the database
    with open(os.path.join('./data', file_name + '.' + file_extension), 'r') as file:
        # Use csv.DictReader to read the file
        csv_reader = csv.DictReader(file, fieldnames=column_names)
        
        # Prepare insert statement. Use ? as placeholders for parameters
        insert_sql = insert_stmt
        
        for row in csv_reader:
            # Retrieve values from the row based on the keys in column_names
            values_to_insert = tuple(row[key] for key in column_names)
            
            # Execute the insert command with the dynamically retrieved values
            cursor.execute(insert_sql, values_to_insert)
            
        # Commit the changes
        connection.commit()

## Create Database and Tables

In [4]:
connection = sqlite3.connect("HR.db")
cursor = connection.cursor()

In [5]:
# Create EMPLOYEES Table  
cursor.execute('''CREATE TABLE IF NOT EXISTS EMPLOYEES (
                            EMP_ID CHAR(9) NOT NULL, 
                            F_NAME VARCHAR(15) NOT NULL,
                            L_NAME VARCHAR(15) NOT NULL,
                            SSN CHAR(9),
                            B_DATE DATE,
                            SEX CHAR,
                            ADDRESS VARCHAR(30),
                            JOB_ID CHAR(9),
                            SALARY DECIMAL(10,2),
                            MANAGER_ID CHAR(9),
                            DEP_ID CHAR(9) NOT NULL,
                            PRIMARY KEY (EMP_ID));''')

# Create JOB_HISTORY Table  
cursor.execute('''CREATE TABLE IF NOT EXISTS JOB_HISTORY (
                            EMPL_ID CHAR(9) NOT NULL, 
                            START_DATE DATE,
                            JOBS_ID CHAR(9) NOT NULL,
                            DEPT_ID CHAR(9),
                            PRIMARY KEY (EMPL_ID,JOBS_ID));''')

# Create JOBS Table  
cursor.execute('''CREATE TABLE IF NOT EXISTS JOBS (
                            JOB_IDENT CHAR(9) NOT NULL, 
                            JOB_TITLE VARCHAR(30),
                            MIN_SALARY DECIMAL(10,2),
                            MAX_SALARY DECIMAL(10,2),
                            PRIMARY KEY (JOB_IDENT));''')

# Create DEPARTMENTS Table  
cursor.execute('''CREATE TABLE IF NOT EXISTS DEPARTMENTS (
                            DEPT_ID_DEP CHAR(9) NOT NULL, 
                            DEP_NAME VARCHAR(15) ,
                            MANAGER_ID CHAR(9),
                            LOC_ID CHAR(9),
                            PRIMARY KEY (DEPT_ID_DEP));''')

# Create LOCATIONS Table  
cursor.execute('''CREATE TABLE IF NOT EXISTS DEPARTMENTS (
                            DEPT_ID_DEP CHAR(9) NOT NULL, 
                            DEP_NAME VARCHAR(15) ,
                            MANAGER_ID CHAR(9),
                            LOC_ID CHAR(9),
                            PRIMARY KEY (DEPT_ID_DEP));''')

In [6]:
# Check table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = cursor.fetchall()

for table in tables:
    print(f"Table Name: {table[0]}")

Table Name: EMPLOYEES
Table Name: JOB_HISTORY
Table Name: JOBS
Table Name: DEPARTMENTS


In [7]:
# Check properties of the tables
for table in tables:
    table_name = table[0]
    print(f"\nProperties of table: {table_name}")
    
    cursor.execute(f"PRAGMA table_info({table_name});")
    
    columns = cursor.fetchall()
    for column in columns:
        cid, name, ctype, notnull, dflt_value, pk = column
        print(f"Column Name: {name}, Type: {ctype}, Not Null: {notnull}, Default Value: {dflt_value}, Primary Key: {pk}")


Properties of table: EMPLOYEES
Column Name: EMP_ID, Type: CHAR(9), Not Null: 1, Default Value: None, Primary Key: 1
Column Name: F_NAME, Type: VARCHAR(15), Not Null: 1, Default Value: None, Primary Key: 0
Column Name: L_NAME, Type: VARCHAR(15), Not Null: 1, Default Value: None, Primary Key: 0
Column Name: SSN, Type: CHAR(9), Not Null: 0, Default Value: None, Primary Key: 0
Column Name: B_DATE, Type: DATE, Not Null: 0, Default Value: None, Primary Key: 0
Column Name: SEX, Type: CHAR, Not Null: 0, Default Value: None, Primary Key: 0
Column Name: ADDRESS, Type: VARCHAR(30), Not Null: 0, Default Value: None, Primary Key: 0
Column Name: JOB_ID, Type: CHAR(9), Not Null: 0, Default Value: None, Primary Key: 0
Column Name: SALARY, Type: DECIMAL(10,2), Not Null: 0, Default Value: None, Primary Key: 0
Column Name: MANAGER_ID, Type: CHAR(9), Not Null: 0, Default Value: None, Primary Key: 0
Column Name: DEP_ID, Type: CHAR(9), Not Null: 1, Default Value: None, Primary Key: 0

Properties of table: 

## EMPLOYEES Table 

In [8]:
table_name, file_name, file_extension = 'EMPLOYEES', 'Employees', 'csv'
insert_data_to_tables(table_name, file_name, file_extension='csv')

In [9]:
# Fetch first two rows of the table and read it in df format
query = 'SELECT * FROM EMPLOYEES LIMIT 5;'
df = pd.read_sql_query(query, connection)
df

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1001,John,Thomas,123456,1976-09-01,M,"5631 Rice, OakPark,IL",100,100000,30001,2
1,E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
2,E1003,Steve,Wells,123458,1980-10-08,M,"291 Springs, Gary,IL",300,50000,30002,5
3,E1004,Santosh,Kumar,123459,1985-07-20,M,"511 Aurora Av, Aurora,IL",400,60000,30002,5
4,E1005,Ahmed,Hussain,123410,1981-04-01,M,"216 Oak Tree, Geneva,IL",500,70000,30001,2


## JOB_HISTORY Table

In [10]:
table_name, file_name, file_extension = 'JOB_HISTORY', 'JobsHistory', 'csv'
insert_data_to_tables(table_name, file_name, file_extension='csv')

In [11]:
# Fetch first two rows of the table and read it in df format
query = 'SELECT * FROM JOB_HISTORY LIMIT 5;'
df = pd.read_sql_query(query, connection)
df

,EMPL_ID,START_DATE,JOBS_ID,DEPT_ID
0,E1001,2000-08-01,100,2
1,E1002,2001-08-01,200,5
2,E1003,2001-08-16,300,5
3,E1004,2000-08-16,400,5
4,E1005,2000-05-30,500,2


## JOBS Table 

In [12]:
table_name, file_name, file_extension = 'JOBS', 'Jobs', 'csv'
insert_data_to_tables(table_name, file_name, file_extension='csv')

In [13]:
# Fetch first two rows of the table and read it in df format
query = 'SELECT * FROM JOBS LIMIT 5;'
df = pd.read_sql_query(query, connection)
df

,JOB_IDENT,JOB_TITLE,MIN_SALARY,MAX_SALARY
0,100,Sr. Architect,60000,100000
1,200,Sr. Software Developer,60000,80000
2,300,Jr.Software Developer,40000,60000
3,400,Jr.Software Developer,40000,60000
4,500,Jr. Architect,50000,70000


## DEPARTMENTS Table

In [14]:
table_name, file_name, file_extension = 'DEPARTMENTS', 'Departments', 'csv'
insert_data_to_tables(table_name, file_name, file_extension='csv')

In [15]:
# Fetch first two rows of the table and read it in df format
query = 'SELECT * FROM DEPARTMENTS LIMIT 5;'
df = pd.read_sql_query(query, connection)
df

,DEPT_ID_DEP,DEP_NAME,MANAGER_ID,LOC_ID
0,2,Architect Group,30001,L0001
1,5,Software Group,30002,L0002
2,7,Design Team,30003,L0003


In [16]:
# Commit changes to save them to the database
connection.commit()

## Some Analysis of the Data 

In [17]:
query = '''SELECT COUNT(DISTINCT EMP_ID) AS 'NUM_EMPLOYEES', 
        COUNT(DISTINCT MANAGER_ID) AS 'NUM_MANAGERS',
        COUNT(DISTINCT DEP_ID) AS 'NUM_DEPARTMENTS' FROM Employees;'''

df = pd.read_sql_query(query, connection)
df

,NUM_EMPLOYEES,NUM_MANAGERS,NUM_DEPARTMENTS
0,10,3,3


In [18]:
query = '''SELECT DISTINCT JOB_TITLE AS 'JOB_TITLES' FROM JOBS;'''

df = pd.read_sql_query(query, connection)
df

,JOB_TITLES
0,Sr. Architect
1,Sr. Software Developer
2,Jr.Software Developer
3,Jr. Architect
4,Lead Architect
5,Jr. Designer
6,Sr. Designer


In [19]:
query = '''SELECT DISTINCT DEP_NAME AS 'DEPARTMENTS' FROM DEPARTMENTS;'''

df = pd.read_sql_query(query, connection)
df

,DEPARTMENTS
0,Architect Group
1,Software Group
2,Design Team


In [20]:
query = '''SELECT F_NAME, L_NAME, ADDRESS
        FROM EMPLOYEES
        WHERE ADDRESS LIKE '%Elgin,IL%' LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,F_NAME,L_NAME,ADDRESS
0,Alice,James,"980 Berry ln, Elgin,IL"
1,Nancy,Allen,"111 Green Pl, Elgin,IL"
2,Ann,Jacob,"111 Britany Springs,Elgin,IL"


In [21]:
query = '''SELECT F_NAME, L_NAME, B_DATE
        FROM EMPLOYEES
        WHERE B_DATE LIKE '197%' LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,F_NAME,L_NAME,B_DATE
0,John,Thomas,1976-09-01
1,Alice,James,1972-07-31
2,Nancy,Allen,1978-06-02
3,Mary,Thomas,1975-05-05


In [22]:
query = '''SELECT * FROM EMPLOYEES
        WHERE (SALARY BETWEEN 60000 AND 70000) AND DEP_ID = 5 LIMIT 5;'''

# Equivalent statement: '''SELECT * FROM EMPLOYEES WHERE (SALARY >= 60000 AND SALARY <= 70000) AND DEP_ID = 5;'''

df = pd.read_sql_query(query, connection)
df

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1004,Santosh,Kumar,123459,1985-07-20,M,"511 Aurora Av, Aurora,IL",400,60000,30002,5
1,E1010,Ann,Jacob,123415,1982-03-30,F,"111 Britany Springs,Elgin,IL",220,70000,30002,5


In [23]:
query = '''SELECT F_NAME, L_NAME, DEP_ID 
        FROM EMPLOYEES
        ORDER BY DEP_ID LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,F_NAME,L_NAME,DEP_ID
0,John,Thomas,2
1,Ahmed,Hussain,2
2,Nancy,Allen,2
3,Alice,James,5
4,Steve,Wells,5


In [24]:
query = '''SELECT F_NAME, L_NAME, DEP_ID 
        FROM EMPLOYEES
        ORDER BY DEP_ID DESC, L_NAME DESC LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,F_NAME,L_NAME,DEP_ID
0,Mary,Thomas,7
1,Andrea,Jones,7
2,Bharath,Gupta,7
3,Steve,Wells,5
4,Santosh,Kumar,5


In [25]:
query = '''SELECT DEP_ID, COUNT(*) AS 'NUM_EMPLOYEES'
        FROM EMPLOYEES
        GROUP BY DEP_ID LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,DEP_ID,NUM_EMPLOYEES
0,2,3
1,5,4
2,7,3


In [26]:
query = '''SELECT DEP_ID, COUNT(*) AS 'NUM_EMPLOYEES', AVG(SALARY) 'AVG_SALARY'
        FROM EMPLOYEES
        GROUP BY DEP_ID
        ORDER BY AVG_SALARY
        LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,DEP_ID,NUM_EMPLOYEES,AVG_SALARY
0,5,4,65000.000000
1,7,3,66666.666667
2,2,3,86666.666667


In [27]:
query = '''SELECT DEP_ID, COUNT(*) AS "NUM_EMPLOYEES", AVG(SALARY) AS "AVG_SALARY"
        FROM EMPLOYEES
        GROUP BY DEP_ID
        HAVING count(*) < 4
        ORDER BY AVG_SALARY
        LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,DEP_ID,NUM_EMPLOYEES,AVG_SALARY
0,7,3,66666.666667
1,2,3,86666.666667


In [28]:
query = '''SELECT F_NAME, L_NAME
        FROM EMPLOYEES
        WHERE F_NAME LIKE 'S%'
        LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,F_NAME,L_NAME
0,Steve,Wells
1,Santosh,Kumar


In [29]:
query = '''SELECT * FROM EMPLOYEES
        ORDER BY B_DATE
        LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
1,E1007,Mary,Thomas,123412,1975-05-05,F,"100 Rose Pl, Gary,IL",650,65000,30003,7
2,E1001,John,Thomas,123456,1976-09-01,M,"5631 Rice, OakPark,IL",100,100000,30001,2
3,E1006,Nancy,Allen,123411,1978-06-02,F,"111 Green Pl, Elgin,IL",600,90000,30001,2
4,E1003,Steve,Wells,123458,1980-10-08,M,"291 Springs, Gary,IL",300,50000,30002,5


In [30]:
query = '''SELECT DEP_ID, AVG(SALARY)
        FROM EMPLOYEES
        GROUP BY DEP_ID
        HAVING AVG(SALARY) >= 60000
        ORDER BY AVG(SALARY) DESC
        LIMIT 5;'''

df = pd.read_sql_query(query, connection)
df

,DEP_ID,AVG(SALARY)
0,2,86666.666667
1,7,66666.666667
2,5,65000.000000


In [31]:
# Close connection
cursor.close()
connection.close()